# Consigna

Link al parcial: https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqu7n114zx

Nintendo of America (EEUU) tiene información de ventas de videojuegos físicas mensuales totalizadas en EEUU las cuales se realizan en cadenas de tiendas de videojuegos en el siguiente RDD: (id_videojuego, id_tienda, mes, anio, total_ventas_mensuales).

Por otro lado tenemos un RDD con información de las tiendas y de su ubicación (id_tienda, direccion, latitud, longitud, codigo_postal, estado).

Con esta información escribir un programa en pySpark para obtener la tienda que realizó menor cantidad de ventas en el estado de “Georgia” en todo el año 2017.

In [1]:
from pyspark import SparkContext

#  Por si hay mas de un contexto de PySpark corriendo (por ejemplo, otro Notebook), esto para utilizar el mismo.
sc = SparkContext.getOrCreate()

In [2]:
#  Vamos a crear algunos datos para poder ver qué es lo que va sucediendo en cada paso de la resolución.
#  El resultado final debería ser la tienda 1, con un total de 35 ventas.

datos_ventas = [
    (0, 1, 2, 2017, 20),
    (2, 2, 3, 2017, 30),
    (1, 1, 7, 2017, 15),
    (0, 1, 3, 2015, 23),
    (3, 3, 1, 2017, 10),
    (3, 3, 8, 2016, 17),
    (3, 3, 6, 2017, 5),
    (0, 4, 10, 2017, 12),
    (15, 2, 5, 2017, 15),
    (0, 5, 1, 2017, 70)
]

datos_tiendas = [
    (1, 'Calle Falsa 123', -3, 4, 1834, 'Georgia'),
    (2, 'Siempre Viva 321', 3, -4, 4381, 'Georgia'),
    (3, 'Av Paseo Colon 850', 1, 2, 1063, 'Texas'),
    (4, 'Sarasa 120', 10, 1, 1234, 'California'),
    (5, 'Mostaza 564', 10, 3, 1324, 'Georgia'),
    (6, 'Ketchup 897', 3, -99, 7891, 'Los Angeles'),
    (4, 'Silver Rathalos 98287', 2, 1, 1234, 'Texas')
]

# Resolución

Básicamente lo que nos piden es filtrar tiendas por año y por estado, luego contar cuántas ventas tiene cada tienda y finalmente quedarnos con la que tenga la menor cantidad.

Notar que en algún punto de la resolución, va a ser necesario hacer un **join** de los RDDs, por lo cual debemos hacer el filtrado de los datos _antes_ de unirlos, para de esta forma hacer menos costosa la unión.

## Paso 1

Cargamos los datos en RDDs. 

Luego filtraremos cada RDD según los criterios que nos piden y descartaremos las columnas que no nos interesan para lo que pide el enunciado.

In [3]:
rdd_ventas = sc.parallelize(datos_ventas)
rdd_tiendas = sc.parallelize(datos_tiendas)

In [4]:
#  Header de rdd_ventas: [id_videojuego, id_tienda, mes, anio, total_ventas_mensuales].
#  Índice de id_tienda: 1.
#  Índice de anio: 3.
#  Índice de total_ventas_mensuales: 4.
#  Filtramos rdd_ventas para quedarnos con aquellos registros cuyo anio sea 2017, luego nos quedamos con tuplas
#  de la forma (id_tienda, total_ventas_mensuales).

rdd_ventas = rdd_ventas.filter(lambda x: x[3] == 2017).map(lambda x: (x[1], x[4]))
rdd_ventas.collect()

[(1, 20), (2, 30), (1, 15), (3, 10), (3, 5), (4, 12), (2, 15), (5, 70)]

In [5]:
#  Header de rdd_tiendas: [id_tienda, direccion, latitud, longitud, codigo_postal, estado].
#  Índice de id_tienda: 0.
#  Índice de estado: 5.
#  Filtramos rdd_tiendas para quedarnos con aquellos registros cuyo estado sea Georgia, luego nos quedamos con tuplas
#  de la forma (id_tienda, estado). Si bien no necesitamos saber el estado, es necesario para poder hacer la unión.

rdd_tiendas = rdd_tiendas.filter(lambda x: x[5] == 'Georgia').map(lambda x: (x[0], x[5]))
rdd_tiendas.collect()

[(1, 'Georgia'), (2, 'Georgia'), (5, 'Georgia')]

## Paso 2

Unimos los RDDs por `id_tienda`, y luego nos quedamos con tuplas de la forma `(id_tienda, total_ventas_mensuales)`.

Observación: El método `join` es un _inner join_ por lo cual se queda con las claves que están presentes en ambos RDD. 

Notar que el resultado es otro RDD el cual está formado por tuplas `(clave, (valor_rdd1, valor_rdd2))`. 

El `map` no es estrictamente necesario, porque de todas formas el `reduceByKey` va a sumar por el elemento numérico de la tupla `(total_ventas_mensuales, estado)` y dejarlas en el mismo formato que obtenemos con el `map`.

In [6]:
rdd_union = rdd_ventas.join(rdd_tiendas).map(lambda x: (x[0], x[1][0]))
rdd_union.collect()

[(1, 20), (1, 15), (2, 30), (2, 15), (5, 70)]

## Paso 3

Finalmente hacemos un `reduceByKey` para obtener las ventas anuales de cada tienda, y luego buscamos la que tenga la menor cantidad de ventas utilizando `reduce`.

Observación: Es importante utilizar `reduce` ya que sólamente nos están pidiendo un único resultado, por lo cual utilizar `takeOrdered` u ordenar de alguna otra forma implicaría una operación innecesariamente costosa.

In [7]:
rdd_union.reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a < b else b)

(1, 35)